In [2]:
import pandas as pd
from data_reciever import datasets
from datetime import datetime
from io import StringIO
import json

Looking for files in: d:\varsity\BTPOS-Sales-Prediction-Model\ds
✓ Loaded: menu_ingredient.JSON
✓ Loaded: date_seasons.csv
✓ Loaded: sales_history.csv
✓ Loaded: date_event.csv
✓ Loaded: date_weekend.csv


In [3]:
date_itemquantity_df=pd.read_csv('../ds/clean/date_itemquantity.csv')
menu_ingredient_df=pd.read_json('../ds/menu_ingredient.json')
item_price_df=pd.read_csv('../ds/clean/date_summary.csv')
sales_history_df=pd.read_csv('../ds/sales_history.csv')

In [31]:
print("xxx weekends check xxx")

date_itemquantity_df['Date'] = pd.to_datetime(date_itemquantity_df['Date'])
date_itemquantity_df['YearMonth'] = date_itemquantity_df['Date'].dt.to_period('M')

weekends_per_month = date_itemquantity_df[date_itemquantity_df['Weekend'] == 1].groupby('YearMonth')['Date'].nunique().reset_index()
weekends_per_month.columns = ['YearMonth', 'Number_of_Weekends']

print("Number of weekends in each month:")
print("=" * 35)
for _, row in weekends_per_month.iterrows():
    print(f"{row['YearMonth']}: {row['Number_of_Weekends']} weekends")

print("\n" + "=" * 35)
print(weekends_per_month.to_string(index=False))

xxx weekends check xxx
Number of weekends in each month:
2025-01: 8 weekends
2025-02: 8 weekends
2025-03: 10 weekends
2025-04: 8 weekends
2025-05: 9 weekends
2025-06: 9 weekends
2025-07: 8 weekends
2025-08: 10 weekends
2025-09: 8 weekends
2025-10: 8 weekends

YearMonth  Number_of_Weekends
  2025-01                   8
  2025-02                   8
  2025-03                  10
  2025-04                   8
  2025-05                   9
  2025-06                   9
  2025-07                   8
  2025-08                  10
  2025-09                   8
  2025-10                   8


In [5]:
print("xxx daily sales xxx")


df_sales = pd.read_csv("../ds/clean/date_itemquantity.csv")
menu_ingredient_df = pd.read_json("../ds/menu_ingredient.json")
df_menu = menu_ingredient_df[["name", "price"]]
df_merged = pd.merge(df_sales, df_menu, left_on="Dish", right_on="name", how="left")
df_merged["sale_amount"] = df_merged["Quantity"] * df_merged["price"]
daily_sales = df_merged.groupby("Date")["sale_amount"].sum().reset_index()
print(daily_sales)
store_avg_sale_day = daily_sales["sale_amount"].sum() / len(daily_sales)
print("Average daily sale:", store_avg_sale_day)


xxx daily sales xxx
           Date  sale_amount
0    2024-10-01         9090
1    2024-10-02         7760
2    2024-10-03         9060
3    2024-10-04         7270
4    2024-10-05        13180
..          ...          ...
390  2025-10-26        14320
391  2025-10-27        12920
392  2025-10-28         9930
393  2025-10-29         8710
394  2025-10-30        13440

[395 rows x 2 columns]
Average daily sale: 9592.075949367088


In [24]:
print ("xxx daily sales check xxx")


df_sales = pd.read_csv("../ds/clean/date_itemquantity.csv")
menu_ingredient_df = pd.read_json("../ds/menu_ingredient.json")


df_menu = menu_ingredient_df[["name", "price"]]


df_merged = pd.merge(df_sales, df_menu, left_on="Dish", right_on="name", how="left")


df_merged["sale_amount"] = df_merged["Quantity"] * df_merged["price"]


daily_sales = df_merged.groupby("Date")["sale_amount"].sum().reset_index()


sales_history_df = pd.read_csv(
    "../ds/sales_history.csv",
    header=None,
    names=["CustomerID", "Date", "Items_Ordered", "Total_Cost_INR", "Season", "isEvent", "Extra"],
    sep=",",
    engine="python",
    on_bad_lines="skip"
)

sales_history_df["Total_Cost_INR"] = pd.to_numeric(sales_history_df["Total_Cost_INR"], errors="coerce")

sales_history_df.dropna(subset=["Date", "Total_Cost_INR"], inplace=True)


daily_recorded = sales_history_df.groupby("Date")["Total_Cost_INR"].sum().reset_index()


comparison = pd.merge(daily_sales, daily_recorded, on="Date", how="inner", suffixes=("_calc", "_recorded"))


comparison["sale_amount_calc"] = pd.to_numeric(comparison["sale_amount"], errors="coerce")
comparison["TotalCost_recorded"] = pd.to_numeric(comparison["Total_Cost_INR"], errors="coerce")


comparison["difference"] = abs(comparison["sale_amount_calc"] - comparison["TotalCost_recorded"])
comparison["accuracy(%)"] = 100 * (1 - (comparison["difference"] / comparison["TotalCost_recorded"]))


print("\n🔹 Daily Comparison between calculated and recorded totals:")
print(comparison.head(10))


avg_accuracy = comparison["accuracy(%)"].mean()
print(f"\n Average matching accuracy: {avg_accuracy:.2f}%")


store_avg_sale_day_history = daily_recorded["Total_Cost_INR"].sum() / len(daily_recorded)
print(f"Average daily sale (from history): {store_avg_sale_day_history:.2f} INR")


store_avg_sale_day_calc = daily_sales["sale_amount"].sum() / len(daily_sales)
print(f"Average daily sale (from calculated): {store_avg_sale_day_calc:.2f} INR")


avg_sale_accuracy = 100 * (1 - abs(store_avg_sale_day_calc - store_avg_sale_day_history) / store_avg_sale_day_history)
print(f"Average sale match accuracy: {avg_sale_accuracy:.2f}%")

if avg_accuracy >= 87:
    avg_sale_day = (store_avg_sale_day_history + store_avg_sale_day_calc) / 2
    print("\n High Accuracy — averaging both sources.")
else:
    avg_sale_day = store_avg_sale_day_history
    print("\n  Low Accuracy — using recorded (history) data only.")

print(f"Final Average Daily Sale: {avg_sale_day:.2f}")

xxx daily sales check xxx

🔹 Daily Comparison between calculated and recorded totals:
         Date  sale_amount  Total_Cost_INR  sale_amount_calc  \
0  2024-10-01         9090          9118.0              9090   
1  2024-10-02         7760          7800.0              7760   
2  2024-10-03         9060          9100.0              9060   
3  2024-10-04         7270          7297.0              7270   
4  2024-10-05        13180         13227.0             13180   
5  2024-10-06        11210         11281.0             11210   
6  2024-10-07        10290         10325.0             10290   
7  2024-10-08         8580          8615.0              8580   
8  2024-10-09         7590          7623.0              7590   
9  2024-10-10         7820          7877.0              7820   

   TotalCost_recorded  difference  accuracy(%)  
0              9118.0        28.0    99.692915  
1              7800.0        40.0    99.487179  
2              9100.0        40.0    99.560440  
3            

In [30]:

print("\n xxx Total orders per day xxx")
daily_order_counts = sales_history_df.groupby("Date").size().reset_index(name="Order_Count")
avg_order_day = daily_order_counts["Order_Count"].mean()
num_days = len(daily_order_counts)

print(daily_order_counts.head())

print(f"\n Total number of days recorded: {num_days}")
print(f" Avg orders/day: {avg_order_day:.2f}")


 xxx Total orders per day xxx
         Date  Order_Count
0  2024-10-01           15
1  2024-10-02           15
2  2024-10-03           16
3  2024-10-04           14
4  2024-10-05           20

 Total number of days recorded: 395
 Avg orders/day: 16.57


In [ ]:
print("model selection")